<a href="https://colab.research.google.com/github/ShakutaiGit/Final_Project_Applied_Data_science/blob/main/Applied_data_science_Flower_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the relevant libraries and define global variables

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental import preprocessing
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model 
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 

input_size = (300,300,3)
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE
size=300
# so we can find the best weights in the model.
cp_callback = tf.keras.callbacks.ModelCheckpoint('best_weights.hdf5', save_best_only=True, save_weights_only=True, verbose=2)


Loading the data and importing the relevant libraries

In [ ]:
train_ds ,train_info= tfds.load('oxford_flowers102', split='train[:80%]', with_info=True)
val_ds,val_info = tfds.load('oxford_flowers102', split='train[80%:90%]', with_info=True)
test_ds , test_info = tfds.load('oxford_flowers102', split='train[90%:]', with_info=True)
# two different  splits 
train_ds1 ,train_info1= tfds.load('oxford_flowers102', split='train[20%:]', with_info=True)
val_ds1,val_info1 = tfds.load('oxford_flowers102', split='train[10%:20%]', with_info=True)
test_ds1 , test_info1 = tfds.load('oxford_flowers102', split='train[:10%]', with_info=True)



Pre Processing of the data- agumentation , resize , normlization 

In [ ]:
# pre processing the data 

# augmentation function 
data_augmentation = tf.keras.Sequential([

            layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
            layers.experimental.preprocessing.RandomRotation(0.2),
            layers.experimental.preprocessing.RandomZoom(0.1)
            #layers.experimental.preprocessing.RandomContrast(factor=0.2)
])
# normlizing and resizing function 
resize_and_rescale = tf.keras.Sequential([
layers.experimental.preprocessing.Resizing(size, size)
])

def prepare(ds):
  # Resize and rescale all kind datasets
  ds = ds.map(lambda data: (resize_and_rescale(data['image']), tf.one_hot(data['label'], 102)), 
              num_parallel_calls=AUTOTUNE)
  # Batch all datasets
  ds = ds.batch(batch_size)
  return ds


ploting the models.

In [ ]:
def show_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()
  

 The typical transfer-learning workflow
 This leads us to how a typical transfer learning workflow can be implemented in Keras:

 Instantiate a base model and load pre-trained weights into it.
 Freeze all layers in the base model by setting trainable = False.
 Create a new model on top of the output of one (or several) layers from the base model.
 Train your new model on your new dataset. 

EfficientNetB3 model

In [ ]:

def eff_net_model(train_ds,val_ds,test_ds):
  print("-------------EfficientNetB3  model--------------")
  train_ds= prepare(train_ds)
  val_ds= prepare(val_ds)
  test_ds= prepare(test_ds)

  inputs = layers.Input(shape=(300, 300, 3))
  x = data_augmentation(inputs)  
  pre_trained_model = keras.applications.EfficientNetB3(include_top=False, weights='imagenet',input_tensor=x,pooling='avg')
  pre_trained_model.trainable= False
  x = layers.BatchNormalization()(pre_trained_model.output)
  top_dropout_rate = 0.5
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = layers.Dense(102, activation="softmax", name="pred")(x)
  model = tf.keras.Model(inputs, outputs, name="ResNet")

  #unfreeze some layers of the pretrained model
  for layer in model.layers[-15:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

  optimizer ='adam'
  model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"]
    )

  model.summary()
  model_info = model.fit(train_ds, epochs=40, callbacks=[cp_callback], validation_data=val_ds)

  print("  Training Accuracy: {}% , Training Loss {}".format(max(model_info.history['acc'])*100,min(model_info.history['loss'])))
  print(" Validation accuracy: {}% , Validation Loss {}".format(max(model_info.history['val_acc'])*100, min(model_info.history['val_loss'])))
  loss, test = model.evaluate(test_ds, verbose=0)
  print("Test accuracy: {}% , Test  loss : {}".format(test * 100,loss))
  return model_info 

ResNet50 model

In [ ]:

def res_net_50_model(train_ds,val_ds,test_ds):
  print("-------------ResNet50 model--------------")
  train_ds= prepare(train_ds)
  val_ds= prepare(val_ds)
  test_ds= prepare(test_ds)


  inputs = layers.Input(shape=(300, 300, 3))
  x = data_augmentation(inputs)  
  pre_trained_model = keras.applications.ResNet50(include_top=False, weights='imagenet',input_tensor=x,pooling='avg')
  pre_trained_model.trainable= False
  #x = layers.GlobalAveragePooling3D(name="avg_pool")(pre_trained_model.output)
  x = layers.BatchNormalization()(pre_trained_model.output)
  top_dropout_rate = 0.5
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = layers.Dense(102, activation="softmax", name="pred")(x)
  model = tf.keras.Model(inputs, outputs, name="ResNet")
  #unfreeze some layers of the pretrained model
  for layer in model.layers[-10:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

  optimizer ='adam'
  model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"]
    )

  model.summary()
  model_info = model.fit(train_ds, epochs=30, callbacks=[cp_callback], validation_data=val_ds)

  print("  Training Accuracy: {}% , Training Loss {}".format(max(model_info.history['acc'])*100,min(model_info.history['loss'])))
  print(" Validation accuracy: {}% , Validation Loss {}".format(max(model_info.history['val_acc'])*100, min(model_info.history['val_loss'])))
  loss, test = model.evaluate(test_ds, verbose=0)
  print("Test accuracy: {}% , Test  loss : {}".format(test * 100,loss))
  return model_info 

In [ ]:
show_eff = eff_net_model(train_ds,val_ds,test_ds)

In [ ]:
show_eff1 = eff_net_model(train_ds1,val_ds1,test_ds1)

In [ ]:
show_res = res_net_50_model(train_ds,val_ds,test_ds)

In [ ]:
show_res1= res_net_50_model(train_ds1,val_ds1,test_ds1)

In [ ]:
show_history(show_eff)

In [ ]:
show_history(show_eff1)

In [ ]:
show_history(show_res)

In [ ]:
show_history(show_res1)